<a href="https://colab.research.google.com/github/alangan17/smazc01/blob/main/SMAZC_01_Homework.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Stock Markets Analytics Zoomcamp Week1

Repo: https://github.com/DataTalksClub/stock-markets-analytics-zoomcamp/tree/main/01-intro-and-data-sources

# Dependencies

In [ ]:
!pip install yfinance

In [ ]:
# IMPORTS
import numpy as np
import pandas as pd

#Fin Data Sources
import yfinance as yf
import pandas_datareader as pdr

#Data viz
import plotly.graph_objs as go
import plotly.express as px

import time
from datetime import date

# Introduction


In this homework, we're going to download finance data from various sources and make simple calculations/analysis.


# Question 1: [Macro] Average growth of GDP in 2023

**What is the average growth (in %) of GDP in 2023?**

Download the timeseries Real Gross Domestic Product (GDPC1) from FRED (https://fred.stlouisfed.org/series/GDPC1).
Calculate year-over-year (YoY) growth rate (that is, divide current value to one 4 quarters ago). Find the average YoY growth in 2023 (average from 4 YoY numbers).
Round to 1 digit after the decimal point: e.g. if you get 5.66% growth => you should answer  5.7

In [ ]:
start = date(year=2022, month=1, day=1)
df1 = pdr.DataReader("GDPC1", "fred", start=start)
df1.reset_index(inplace=True)
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8 entries, 0 to 7
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   DATE    8 non-null      datetime64[ns]
 1   GDPC1   8 non-null      float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 256.0 bytes


In [ ]:
df1['gdp_growth_yoy'] = df1.GDPC1/df1.GDPC1.shift(4)-1
df1.tail(8)

,DATE,GDPC1,gdp_growth_yoy
0,2022-01-01,21738.871,NaN
1,2022-04-01,21708.160,NaN
2,2022-07-01,21851.134,NaN
3,2022-10-01,21989.981,NaN
4,2023-01-01,22112.329,0.017179
5,2023-04-01,22225.350,0.023825
6,2023-07-01,22490.692,0.029269
7,2023-10-01,22679.255,0.031345


In [ ]:
df1_2023 = df1[df1['DATE'].dt.year == 2023]
average_gdpc1_us_yoy_2023 = round(df1_2023['gdp_growth_yoy'].mean() * 100, 1)
print(f"Average Year to Year GDP Growth in 2023: {average_gdpc1_us_yoy_2023}%")

Average Year to Year GDP Growth in 2023: 2.5%


# Question 2. [Macro] Inverse "Treasury Yield"

*Find the min value of (dgs10-dgs2) after since year 2000 (2000-01-01) and write it down as an answer, round to 1 digit after the decimal point.**


Download DGS2 and DGS10 interest rates series (https://fred.stlouisfed.org/series/DGS2,
 https://fred.stlouisfed.org/series/DGS10). Join them together to one dataframe on date (you might need to read about pandas.DataFrame.join()), calculate the difference dgs10-dgs2 daily.

(Additional: think about what does the "inverted yield curve" mean for the market and investors? do you see the same thing in your country/market of interest? Do you think it can be a good predictive feature for the models?)


In [ ]:
start = date(year=2000, month=1, day=1)
# Market Yield on U.S. Treasury Securities at 2-Year Constant Maturity
df2_dgs2 = pdr.DataReader("DGS2", "fred", start=start)
df2_dgs2.reset_index(inplace=True)
df2_dgs2.info()

# Market Yield on U.S. Treasury Securities at 10-Year Constant Maturity
df2_dgs10 = pdr.DataReader("DGS10", "fred", start=start)
df2_dgs10.reset_index(inplace=True)
df2_dgs10.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6339 entries, 0 to 6338
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   DATE    6339 non-null   datetime64[ns]
 1   DGS2    6078 non-null   float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 99.2 KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6339 entries, 0 to 6338
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   DATE    6339 non-null   datetime64[ns]
 1   DGS10   6078 non-null   float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 99.2 KB


In [ ]:
df2 = pd.merge(df2_dgs2, df2_dgs10, on='DATE', how='inner')
df2['dgs10-dgs2'] = round(df2['DGS10'] - df2['DGS2'], 1)
df2

,DATE,DGS2,DGS10,dgs10-dgs2
0,2000-01-03,6.38,6.58,0.2
1,2000-01-04,6.30,6.49,0.2
2,2000-01-05,6.38,6.62,0.2
3,2000-01-06,6.35,6.57,0.2
4,2000-01-07,6.31,6.52,0.2
...,...,...,...,...
6334,2024-04-12,4.88,4.50,-0.4
6335,2024-04-15,4.93,4.63,-0.3
6336,2024-04-16,4.97,4.67,-0.3
6337,2024-04-17,4.93,4.59,-0.3


In [ ]:
df2[df2['dgs10-dgs2'] == df2['dgs10-dgs2'].min()]

,DATE,DGS2,DGS10,dgs10-dgs2
6047,2023-03-08,5.05,3.98,-1.1
6129,2023-06-30,4.87,3.81,-1.1
6130,2023-07-03,4.94,3.86,-1.1


# Question 3. [Index] Which Index is better recently?

**Compare S&P 500 and IPC Mexico indexes by the 5 year growth and write down the largest value as an answer (%)**

Download on Yahoo Finance two daily index prices for S&P 500 (^GSPC, https://finance.yahoo.com/quote/%5EGSPC/) and IPC Mexico (^MXX, https://finance.yahoo.com/quote/%5EMXX/). Compare 5Y growth for both (between 2019-04-09 and 2024-04-09). Select the higher growing index and write down the growth in % (closest integer %). E.g. if ratio end/start was 2.0925 (or growth of 109.25%), you need to write down 109 as your answer.

(Additional: think of other indexes and try to download stats and compare the growth? Do create 10Y and 20Y growth stats. What is an average yearly growth rate (CAGR) for each of the indexes you select?)

# Question 4. [Stocks OHLCV] 52-weeks range ratio (2023) for the selected stocks

**Find the largest range ratio [=(max-min)/max] of Adj.Close prices in 2023**


Download the 2023 daily OHLCV data on Yahoo Finance for top6 stocks on earnings (https://companiesmarketcap.com/most-profitable-companies/): 2222.SR,BRK-B, AAPL, MSFT, GOOG, JPM.

Here is the example data you should see in Pandas for "2222.SR": https://finance.yahoo.com/quote/2222.SR/history

Calculate maximum-minimim "Adj.Close" price for each stock and divide it by the maximum "Adj.Close" value.
Round the result to two decimal places (e.g. 0.1575 will be 0.16)

(Additional: why this may be important for your research?)

# Question 5. [Stocks] Dividend Yield

**Find the largest dividend yield for the same set of stocks**

Use the same list of companies (2222.SR,BRK-B, AAPL, MSFT, GOOG, JPM) and download all dividends paid in 2023.
You can use `get_actions()` method or `.dividends` field in yfinance library (https://github.com/ranaroussi/yfinance?tab=readme-ov-file#quick-start)

Sum up all dividends paid in 2023 per company and divide each value by the closing price (Adj.Close) at the last trading day of the year.

Find the maximm value in % and round to 1 digit after the decimal point. (E.g., if you obtained $1.25 dividends paid and the end year stock price is $100, the dividend yield is 1.25% -- and your answer should be equal to 1.3)

# Question 6. [Exploratory] Investigate new metrics

Download and explore a few additional metrics or time series that might be valuable for your project and write down why (briefly).

# Question 7. [Exploratory] Time-driven strategy description around earnings releases

Explore earning dates for the whole month of April - e.g. using YahooFinance earnings calendar (https://finance.yahoo.com/calendar/earnings?from=2024-04-21&to=2024-04-27&day=2024-04-23). Compare with the previous closed earnings (e.g., recent dates with full data https://finance.yahoo.com/calendar/earnings?from=2024-04-07&to=2024-04-13&day=2024-04-08).

Describe an analytical strategy/idea (you're not required to implement it) to select a subset companies of interest based on the future events data.